In [2]:
# Install required packages

!pip install "ibm-watsonx-ai==1.1.2"
!pip install "langchain==0.2.1"
!pip install "langchain-ibm==0.1.11"
!pip install "langchain-community==0.2.1"
!pip install "chromadb==0.4.24"
!pip install "pypdf==4.3.1"
!pip install "lark==1.1.9"
!pip install 'posthog<6.0.0'

  Using cached ibm_cos_sdk-2.13.6-py3-none-any.whl
  Using cached ibm_cos_sdk_core-2.13.6-py3-none-any.whl
  Using cached ibm_cos_sdk_s3transfer-2.13.6-py3-none-any.whl
   ---------------------------------------- 0.0/966.6 kB ? eta -:--:--
   --------------------- ------------------ 524.3/966.6 kB 8.5 MB/s eta 0:00:01
   ---------------------------------------- 966.6/966.6 kB 5.0 MB/s  0:00:00

  Attempting uninstall: ibm-cos-sdk-core

    Found existing installation: ibm-cos-sdk-core 2.14.1

    Uninstalling ibm-cos-sdk-core-2.14.1:

      Successfully uninstalled ibm-cos-sdk-core-2.14.1

   ---------------------------------------- 0/4 [ibm-cos-sdk-core]
   ---------------------------------------- 0/4 [ibm-cos-sdk-core]
   ---------------------------------------- 0/4 [ibm-cos-sdk-core]
   ---------------------------------------- 0/4 [ibm-cos-sdk-core]
   ---------------------------------------- 0/4 [ibm-cos-sdk-core]
   ---------------------------------------- 0/4 [ibm-cos-sdk-core]
 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-ibm 0.1.4 requires ibm-watsonx-ai<0.3.0,>=0.2.6, but you have ibm-watsonx-ai 1.1.2 which is incompatible.
langchain-ibm 0.1.4 requires langchain-core<0.2.0,>=0.1.42, but you have langchain-core 0.2.43 which is incompatible.


   ---------------------------------------- 0.0/973.5 kB ? eta -:--:--
   -------------------------------- ------- 786.4/973.5 kB 4.8 MB/s eta 0:00:01
   ---------------------------------------- 973.5/973.5 kB 5.0 MB/s  0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.11
    Uninstalling langchain-0.2.11:
      Successfully uninstalled langchain-0.2.11


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.2.10 requires langchain<0.3.0,>=0.2.9, but you have langchain 0.2.1 which is incompatible.


  Attempting uninstall: langchain-ibm
    Found existing installation: langchain-ibm 0.1.4
    Uninstalling langchain-ibm-0.1.4:
      Successfully uninstalled langchain-ibm-0.1.4
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   -------------- ------------------------- 0.8/2.1 MB 3.7 MB/s eta 0:00:01
   ----------------------------- ---------- 1.6/2.1 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 3.1 MB/s  0:00:00
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.2.10
    Uninstalling langchain-community-0.2.10:
      Successfully uninstalled langchain-community-0.2.10
  Using cached chromadb-0.4.24-py3-none-any.whl.metadata (7.3 kB)
Using cached chromadb-0.4.24-py3-none-any.whl (525 kB)
  Attempting uninstall: chromadb
    Found existing installation: chromadb 1.0.12
    Uninstalling chromadb-1.0.12:
      Success

The system cannot find the file specified.


In [3]:
# Supress warnings

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

### Creating a retriever model

#### The following steps are involved to create a retriever model using LangChain:

• Building LLMs

• Splitting documents into chunks

• Building an embedding model

• Retrieving related knowledge from text

In [6]:
# Build the LLM

from dotenv import load_dotenv
import os
from langchain.chat_models import ChatOpenAI

# Load API key from .env
load_dotenv()
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
if not openrouter_api_key:
    raise ValueError("OPENROUTER_API_KEY not found in environment variables")

# Initialize LLM via OpenRouter
llm = ChatOpenAI(
    model="mistralai/mistral-small-3.1-24b-instruct:free",
    api_key=openrouter_api_key,
    base_url="https://openrouter.ai/api/v1",
    temperature=0.2,
)


# Test LLM connection
try:
    response = llm.invoke("Hello, can you respond?")
    print("LLM connection successful:", response.content)
except Exception as e:
    print("LLM connection failed:", str(e))


LLM connection successful: Yes, I can respond. How can I assist you today?


In [ ]:
# Use the Text Splitter

from langchain.text_splitter import RecursiveCharacterTextSplitter

def text_splitter(data, chunk_size, chunk_overlap):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    chunks = text_splitter.split_documents(data)
    return chunks


In [18]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Create the Embedding Model

HFembeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
# Download the sample document

# !curl "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/MZ9z1lm-Ui3YBp3SYWLTAQ/companypolicies.txt" > companypolicies.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 15660  100 15660    0     0   9339      0  0:00:01  0:00:01 --:--:--  9349


In [ ]:
# Use Text Loader to load the document

from langchain.document_loaders import TextLoader

loader = TextLoader("companypolicies.txt")
txt_data = loader.load()

# txt_data

1

In [16]:
# Split txt_data into chunks. chunk_size = 200, chunk_overlap = 20 has been set.
chunks_txt = text_splitter(txt_data, chunk_size=200, chunk_overlap=20)

In [19]:
# Store the embeddings to ChromaDB

from langchain.vectorstores import Chroma

# Create and store the embeddings in ChromaDB
vectordb = Chroma.from_documents(documents=chunks_txt, embedding=HFembeddings)

In [23]:
# Simple similarity search
query = "email policy"
# retriever = vectordb.as_retriever(search_kwargs={"k":1}) # to limit number of docs retrieved
retriever = vectordb.as_retriever()

docs = retriever.invoke(query)

docs

[Document(metadata={'source': 'companypolicies.txt'}, page_content='3.\tInternet and Email Policy'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Our Internet and Email Policy aims to promote safe, responsible usage of digital communication tools that align with our values and legal obligations. Each employee is expected to understand and'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Our Internet and Email Policy is established to guide the responsible and secure use of these essential tools within our organization. We recognize their significance in daily business operations and'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Confidentiality: Reserve email for the transmission of confidential information, trade secrets, and sensitive customer data only when encryption is applied. Exercise discretion when discussing')]

In [24]:
# MAXIMUM MARGINAL RELEVANCE (MMR) RETRIEVER
# retriever_mmr = vectordb.as_retriever(search_type="mmr", search_kwargs={"k":3, "fetch_k":5})

retriever_mmr = vectordb.as_retriever(search_type="mmr")
docs_mmr = retriever_mmr.invoke(query)
docs_mmr

[Document(metadata={'source': 'companypolicies.txt'}, page_content='3.\tInternet and Email Policy'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Confidentiality: Reserve email for the transmission of confidential information, trade secrets, and sensitive customer data only when encryption is applied. Exercise discretion when discussing'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Review of Policy: This policy will be reviewed periodically to ensure its alignment with evolving legal requirements and best practices for maintaining a healthy and safe workplace.'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='individual found to be in violation of this policy.')]

In [ ]:
# Similarity Score Threshold Retriever
# You can also set a retrieval method that defines a similarity score threshold, returning only documents with a score above that threshold.

retriever = vectordb.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.4}
)

docs = retriever.invoke(query)
docs

[Document(metadata={'source': 'companypolicies.txt'}, page_content='3.\tInternet and Email Policy'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Our Internet and Email Policy aims to promote safe, responsible usage of digital communication tools that align with our values and legal obligations. Each employee is expected to understand and'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Our Internet and Email Policy is established to guide the responsible and secure use of these essential tools within our organization. We recognize their significance in daily business operations and'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Confidentiality: Reserve email for the transmission of confidential information, trade secrets, and sensitive customer data only when encryption is applied. Exercise discretion when discussing')]

In [ ]:
# Multi-Query Retriever
# Distance-based vector database retrieval represents queries in high-dimensional space and finds similar embedded documents based on "distance". 
# However, retrieval results may vary with subtle changes in query wording or if the embeddings do not accurately capture the data's semantics.

# The `MultiQueryRetriever` addresses this by using an LLM to generate multiple queries from different perspectives for a given user input query. 
# For each query, it retrieves a set of relevant documents and then takes the unique union of these results to form a larger set of potentially relevant documents. 
# By generating multiple perspectives on the same question, the `MultiQueryRetriever` can potentially overcome some limitations of distance-based retrieval, resulting in a richer and more diverse set of results.


# A PDF document has been prepared to demonstrate this Multi-Query Retriever.
from langchain_community.document_loaders import PyPDFLoader


loader = PyPDFLoader("langchain_paper.pdf")
pdf_data = loader.load()

# pdf_data[1]

Document(metadata={'source': 'langchain_paper.pdf', 'page': 1}, page_content='LangChain helps us to unlock the ability to harness the \nLLM’s immense potential in tasks such as document analysis, \nchatbot development, code analysis, and countless other \napplications. Whether your desire is to unlock deeper natural \nlanguage understanding , enhance data, or circumvent \nlanguage barriers through translation, LangChain is ready to \nprovide the tools and programming support you need to do \nwithout it that it is not only difficult but also fresh for you . Its \ncore functionalities encompass:  \n1. Context -Aware Capabilities: LangChain facilitates the \ndevelopment of applications that are inherently \ncontext -aware. This means that these applications can \nconnect to a language model and draw from various \nsources of context, such as prompt instructions, a  few-\nshot examples, or existing content, to ground their \nresponses effectively.  \n2. Reasoning Abilities: LangChain equip

In [ ]:
# Split the document and store the embeddings into a vector database.

# Split
chunks_pdf = text_splitter(pdf_data, chunk_size=500, chunk_overlap=20)

# Store in Vector DB (ChromaDB)
ids = vectordb.get()["ids"]
# ids
# vectordb.delete(ids=ids) # You must delete existing entries before adding new ones with same IDs

vectordb = Chroma.from_documents(documents=chunks_pdf, embedding=HFembeddings)

In [ ]:
# vectordb.get()["ids"]  # Verify new entries have been added

In [43]:
# Multi-Query Retriever
from langchain.retrievers.multi_query import MultiQueryRetriever

query = "What does the paper say about langchain?"

retriever = MultiQueryRetriever.from_llm(
    llm=llm,
    retriever=vectordb.as_retriever()
)

In [44]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [45]:
docs = retriever.invoke(query)
docs

INFO:langchain.retrievers.multi_query:Generated queries: ['What information does the paper provide regarding LangChain?', '', 'What are the insights on LangChain mentioned in the paper?', '', 'What details about LangChain are discussed in the paper?']


[Document(metadata={'page': 5, 'source': 'langchain_paper.pdf'}, page_content='[11] LangChain’s Large Language Model Chain,  \nhttps://python.langchain.com/docs/modules/chains/foundational/llm_c\nhain (accessed Nov. 29, 2023).  \n[12] Streamlit, https://streamlit.io/  (accessed Nov. 29, 2023).'),
 Document(metadata={'page': 1, 'source': 'langchain_paper.pdf'}, page_content='LangChain helps us to unlock the ability to harness the \nLLM’s immense potential in tasks such as document analysis, \nchatbot development, code analysis, and countless other \napplications. Whether your desire is to unlock deeper natural \nlanguage understanding , enhance data, or circumvent \nlanguage barriers through translation, LangChain is ready to \nprovide the tools and programming support you need to do \nwithout it that it is not only difficult but also fresh for you . Its'),
 Document(metadata={'page': 5, 'source': 'langchain_paper.pdf'}, page_content='Nov. 29, 2023).  [7] LangChain’s Prompt, https://pyt

In [ ]:
# Self-Querying Retriever
# A Self-Querying Retriever, as the name suggests, has the ability to query itself. 
# Specifically, given a natural language query, the retriever uses a query-constructing LLM chain to generate a structured query. It then applies this structured query to its underlying vector store. 
# This enables the retriever to not only use the user-input query for semantic similarity comparison with the contents of stored documents but also to extract and apply filters based on the metadata of those documents.

In [46]:
# Demonstration of Self-Querying Retriever

from langchain_core.documents import Document
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from lark import lark

In [47]:
# Docs for demonstration

docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]

In [ ]:
# Now you can instantiate your retriever. 
# To do this, you'll need to provide some upfront information about the metadata fields that your documents support and a brief description of the document contents.

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]

In [50]:
vectordb = Chroma.from_documents(documents=docs, embedding=HFembeddings)

In [ ]:
# Use the Self-Querying Retriever

document_content_description = "Brief summary of a movie."

retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectordb,
    document_content_description=document_content_description,
    metadata_field_info=metadata_field_info,
    document_contents="page_content",
)

SelfQueryRetriever(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000168540A6A90>, query_constructor=RunnableBinding(bound=FewShotPromptTemplate(input_variables=['query'], examples=[{'i': 1, 'data_source': '```json\n{{\n    "content": "Lyrics of a song",\n    "attributes": {{\n        "artist": {{\n            "type": "string",\n            "description": "Name of the song artist"\n        }},\n        "length": {{\n            "type": "integer",\n            "description": "Length of the song in seconds"\n        }},\n        "genre": {{\n            "type": "string",\n            "description": "The song genre, one of "pop", "rock" or "rap""\n        }}\n    }}\n}}\n```', 'user_query': 'What are songs by Taylor Swift or Katy Perry about teenage romance under 3 minutes long in the dance pop genre', 'structured_request': '```json\n{{\n    "query": "teenager love",\n    "filter": "and(or(eq(\\"artist\\", \\"Taylor Swift\\"), eq(\\"artist\\", \\"Katy Perry\\"))

In [56]:
# retriever.invoke("Find sci-fi movies released before 2000 with rating over 7")

# This example only specifies a filter
retriever.invoke("I want to watch a movie rated higher than 8.5")

[Document(metadata={'director': 'Andrei Tarkovsky', 'genre': 'thriller', 'rating': 9.9, 'year': 1979}, page_content='Three men walk into the Zone, three men walk out of the Zone'),
 Document(metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006}, page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea')]

In [57]:
# This example specifies a query and a filter
retriever.invoke("Has Greta Gerwig directed any movies about women")

[Document(metadata={'director': 'Greta Gerwig', 'rating': 8.3, 'year': 2019}, page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them')]

In [59]:
# This example specifies a composite filter
retriever.invoke("What's a highly rated (above 8.5) science fiction film?")

[Document(metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006}, page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea'),
 Document(metadata={'director': 'Andrei Tarkovsky', 'genre': 'thriller', 'rating': 9.9, 'year': 1979}, page_content='Three men walk into the Zone, three men walk out of the Zone')]

In [ ]:
# Parent Document Retriever
# When splitting documents for retrieval, there are often conflicting desires:

# You may want to have small documents so that their embeddings can most accurately reflect their meaning. 
# If the documents are too long, the embeddings can lose meaning.

# You want to have long enough documents so that the context of each chunk is retained.
# The `ParentDocumentRetriever` strikes that balance by splitting and storing small chunks of data. During retrieval, it first fetches the small chunks but then looks up the parent IDs for those chunks and returns those larger documents.

from langchain.retrievers import ParentDocumentRetriever
from langchain_text_splitters import CharacterTextSplitter
from langchain.storage import InMemoryStore


In [63]:
# Set two splitters. One is with big chunk size (parent) and one is with small chunk size (child)
parent_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=20, separator='\n')
child_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20, separator='\n')

In [65]:
vectordb = Chroma(
    collection_name="splitter_parents", embedding_function=HFembeddings
)

#vectordb = Chroma.from_documents(documents=chunks_pdf, embedding=watsonx_embedding())

# The storage layer for the parent documents
store = InMemoryStore()

In [66]:
retriever = ParentDocumentRetriever(
    vectorstore=vectordb,
    docstore=store,
    parent_splitter=parent_splitter,
    child_splitter=child_splitter,
)

In [67]:
retriever.add_documents(txt_data)

In [68]:
len(list(store.yield_keys()))

19

In [69]:
sub_docs = vectordb.similarity_search("smoking policy")

In [77]:
sub_docs[0]
# print(sub_docs[0].page_content)

Document(metadata={'doc_id': '507dde79-8e9d-4866-bd0a-a40816070b9f', 'source': 'companypolicies.txt'}, page_content='5.\tSmoking Policy')

In [75]:
retrieved_docs = retriever.invoke("smoking policy")
print(retrieved_docs[0].page_content)

5.	Smoking Policy
Policy Purpose: The Smoking Policy has been established to provide clear guidance and expectations concerning smoking on company premises. This policy is in place to ensure a safe and healthy environment for all employees, visitors, and the general public.
Designated Smoking Areas: Smoking is only permitted in designated smoking areas, as marked by appropriate signage. These areas have been chosen to minimize exposure to secondhand smoke and to maintain the overall cleanliness of the premises.
Smoking Restrictions: Smoking inside company buildings, offices, meeting rooms, and other enclosed spaces is strictly prohibited. This includes electronic cigarettes and vaping devices.
Compliance with Applicable Laws: All employees and visitors must adhere to relevant federal, state, and local smoking laws and regulations.


In [81]:
# Self-Query Retriever with a Filter

vectordb = Chroma.from_documents(docs, embedding=HFembeddings)

retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    document_content_description=document_content_description,
    metadata_field_info=metadata_field_info,
    vectorstore=vectordb,
    document_contents="page_content",
    
)

# This example specifies a query with filter
retriever.invoke(
    "I want to watch a movie directed by Christopher Nolan"
)

[Document(metadata={'director': 'Christopher Nolan', 'rating': 8.2, 'year': 2010}, page_content='Leo DiCaprio gets lost in a dream within a dream within a dream within a ...'),
 Document(metadata={'director': 'Christopher Nolan', 'rating': 8.2, 'year': 2010}, page_content='Leo DiCaprio gets lost in a dream within a dream within a dream within a ...'),
 Document(metadata={'director': 'Christopher Nolan', 'rating': 8.2, 'year': 2010}, page_content='Leo DiCaprio gets lost in a dream within a dream within a dream within a ...'),
 Document(metadata={'director': 'Christopher Nolan', 'rating': 8.2, 'year': 2010}, page_content='Leo DiCaprio gets lost in a dream within a dream within a dream within a ...')]